In [ ]:
%pip install openai load_dotenv

In [ ]:
# openai 패키지의 버전 정보 확인
%pip show openai | grep Version

In [ ]:
import os 
from dotenv import load_dotenv
import openai
import json 
from openai import OpenAI


load_dotenv()


print(openai.__version__)

def show_json(obj):
    #obj의 모델을 Json 형태로 변환 후 출력
    display(json.loads(obj.model_dump_json()))


api_key = os.environ.get('OPENAI_API_KEY')
# Open ai API를 사용하기 위한 클라이언트 객체생성
client = OpenAI(api_key=api_key)


In [ ]:

assistant = client.beta.assistants.create(
    name = "상표 식별력 판단 AI(GPT-4o)",
    instructions="""
    1. Role
    상표 식별력 판단 AI 서비스는 업로드한 파일들을 Retrieval 하고, 상표의 이미지 또는 텍스트를 분석하여, 해당 상표의 식별력을 평가하고 등록 가능성을 판단하는 역할을 수행합니다.
""",
model='gpt-4o',

)
# 생성된 챗봇의 정보를 json 형태로 출력 
show_json(assistant)


### ASSISTANT 삭제

In [ ]:
assistant_delete = client.beta.assistants.delete(
    assistant_id = 'asst_x80ooPyJACW7qsswAChpQyMG'
)

In [ ]:
# 기존 어시스턴트 ID 확인
assistant_list = client.beta.assistants.list()

for assistant in assistant_list:
    print(f"Assistant Name: {assistant.name}, Assistant ID: {assistant.id}")


In [ ]:
# assistant id를 별도의 변수에 담음
ASSISTANT_ID = 'asst_GDlNLfM4j2LCpTYFgULSR1V6'

print(f"[새로 생성한 ASSISTANT_ID]\n{ASSISTANT_ID}")


In [ ]:
# assistant 삭제
client.beta.assistants.delete(assistant_id='asst_GDlNLfM4j2LCpTYFgULSR1V6')
# print(f"어시스턴트 {ASSISTANT_ID}가 삭제되었습니다.")


# 업데이트 및 Code_interpreter
 assistant가 자체적으로 구동이 가능한 코드를 구현하고 query를 만들어 검색
- 데이터가 많을 경우 토큰을 효율적으로 사용가능

In [ ]:
# update assistant
assistant = client.beta.assistants.update(
    ASSISTANT_ID,
    tools=[{'type':'code_interpreter'}], 
)
show_json(assistant)

In [ ]:
def upload_files(files):
    uploaded_files = []
    for filepath in files:
        file = client.beta.create(
            file = open(
                filepath,
                'rb'
            ),
            purpose='assistants'
        )
        uploaded_files.append(file.id)
        print(f'[업로드한 파일 ID]\n{file.id}')
    return uploaded_files


In [ ]:

#업로드할 파일들의 경로를 지정
files_to_uploaded = [
    '_docs/example/[의견서예시][의견서미제출-거절]선행상표조사_MindShare.pdf',
    '_docs/example/[의견서예시][의견서제출-거절]crople선행상표조사보고서(제출).pdf',
    '_docs/example/[의견서예시]몸선필라테스&발레핏.pdf',
    '_docs/example/상표검색 프로세스.pdf',
    '_docs/example/상표심사기준202405.pdf',
    '_docs/example/선행상표조사결과(샘플)_화음이 만든 샘플임_240822.pdf',
    '_docs/example/상표유사여부보고서(별책).pdf',

]

file_streams = [open(path, 'rb') for path in files_to_uploaded]


### 파일 백터화

In [ ]:
# vector_store = client.beta.vector_stores.create(
#     name = '상표 식별 documents',
# )

vector_store = client.beta.vector_stores.update(
    vector_store_id= 'vs_lnyjqbRPhkqR5RkQ3Y3pdiN1'
)
show_json(vector_store)

In [ ]:
client.beta.vector_stores.list()

In [ ]:
client.files.delete(
    file_id=
        'file-hpjBTNyZRMpQG4lxlqGLaa2c'
    
)

### 업로드된 파일 리스트

In [ ]:
for file in client.files.list():
    print(f'[파일 ID] {file.id} [파일명]{file.filename}')

### 2. 파일을 업로드 하고 vector store에 추가

In [ ]:
# 파일 업로드 및 백터 스토어에 추가
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files = file_streams
)

### 백터스토어 리스트

In [6]:
## 벡터스토어 리스트 검색 ###

vector_store_list = client.beta.vector_stores.list()

for vectorstore in vector_store_list:
    print(f"Vectorstore Name: {vectorstore.name}, Vectorstore ID: {vectorstore.id}")


Vectorstore Name: 의견서 작성 예시, Vectorstore ID: vs_I1f6CEXf49Ul7Ko6iOx5oeQ8
Vectorstore Name: Similarity Code Batch Ver., Vectorstore ID: vs_0dJoKkouQ6Qa7HbczRnwC1VG
Vectorstore Name: Similarity Code, Vectorstore ID: vs_m1b79x1RYgGhA6f8qaUIHzUU
Vectorstore Name: Vienna Code, Vectorstore ID: vs_QtSniSyyBiMQ8P8AJYONaBJq
Vectorstore Name: 상표 식별 documents, Vectorstore ID: vs_rLXYrSoCNE7aNpLI6cBGPseN


In [ ]:
# Upload the user provided file to OpenAI
message_file = client.files.create(
  file=open("edgar/aapl-10k.pdf", "rb"), purpose="assistants"
)
 
# Create a thread and attach the file to the message
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "How many shares of AAPL were outstanding at the end of of October 2023?",
      # Attach the new file to the message.
      "attachments": [
        { "file_id": message_file.id, "tools": [{"type": "file_search"}] }
      ],
    }
  ]
)
 
# The thread now has a vector store with that file in its tool resources.
print(thread.tool_resources.file_search)

### 파일 청크 검사 
파일 검색 결과의 품질을 개선하기 위한 첫 번째 단계는 어시스턴트의 현재 동작을 검사하는 것입니다. 
대부분의 경우, 이는 성능이 좋지 않은 어시스턴트의 응답을 조사하는 것을 포함합니다. 
REST API를 사용하여 과거 실행 단계에 대한 세부 정보를 얻을 수 있으며 , 
특히 include쿼리 매개변수를 사용하여 결과를 생성하는 데 사용되는 파일 청크를 가져올 수 있습니다.

In [ ]:
from openai import OpenAI
client = OpenAI()

run_step = client.beta.threads.runs.steps.retrieve(
    thread_id="thread_abc123",
    run_id="run_abc123",
    step_id="step_abc123",
    include=["step_details.tool_calls[*].file_search.results[*].content"]
)

print(run_step)

### Vector Store 삭제

In [ ]:

## vectorstore 삭제 ###
vector_store = client.beta.vector_stores.delete(
    vector_store_id='vs_ZOgnvSj623UBSYn3sdcRU3c0'
)

### 3. 새 백터 스토어를 사용하도록 어시스턴트 업데이트

In [ ]:
# 업로드된 파일 참조 및 검색 요청
assistant = client.beta.assistants.update(
    assistant_id=ASSISTANT_ID,
    tool_resources={'file_search': {'vector_store_ids': [vector_store.id]}},
    temperature=0
)
show_json(assistant)

### 4. 스레드 만들기
스레드에 메시지 첨부파일로 파일을 첨부 가능
vector_store 스레드와 연관된 다른 파일이 생성되거나, 스레드에 이미 벡터 스토어가 첨부 되어있는 경우 새 파일을 기존 스레드 벡터 스토어에 첨부


	1.	파일 업로드: message_file 객체는 업로드된 이미지의 파일 ID를 가지고 있으며, 이 파일 ID를 스레드에서 사용하여 이미지 파일을 참조합니다.
	2.	스레드 생성: thread 객체는 대화의 흐름을 관리하는 스레드이며, 여기에 사용자가 보낸 메시지(텍스트와 이미지)가 포함됩니다.
	3.	어시스턴트 실행: client.beta.threads.runs.create()를 사용하여 스레드 내에서 어시스턴트를 실행시킵니다. 이때 어시스턴트의 ID가 필요하며, 해당 어시스턴트가 이미지를 분석할 수 있는지 확인해야 합니다.
	4.	결과 확인: client.beta.threads.messages.list()로 스레드의 대화 내용을 확인하여 어시스턴트가 이미지에 대해 어떤 응답을 했는지 확인할 수 있습니다.


In [ ]:
thread = client.beta.threads.create()

show_json(thread)


In [ ]:
message = client.beta.threads.messages.create(
    thread_id= thread.id,
    role='user',
    content='식별력이 뭐야?'
)
show_json(message)

In [ ]:
run = client.beta.threads.runs.create(
    thread_id=thread.id, # 생성한스레드(카톡방)
    assistant_id=ASSISTANT_ID # 적용할 AssistantID
)
show_json(run)

run을 생성하는 것은 비동기 작업이다.
이는 Run의 메타데이터와 함께 즉시 반환되며, status 는 queued(대기중)으로 표기된다.
status 는 Assistant가 작업을 수행함에 따라(도구 사용 및 메시지 추가와 같은) 업데이트될 것임.

### status 목록
- queued : 아직 실행이 되지 않고 대기중인 상태
- in_progress: 처리중
- requires_action : 사용자 입력 대기중
- cancelling : 작업 취소중
- cancelled : 취소 완료
- failed: 오류(실패)
- completed: 작업완료
- expired: 작업 만료

In [ ]:
import time 

def wait_on_run(run, thread):
    # 주어진 실행 (run)이 완료 될때까지 대기
    # status 가 'queued' 또는 'inprogress'인 경우에는 계속 polling 하며 대기
    while run.status == 'queued' or run.status == 'in_progress':
        # run.status를 업데이트 합니다.
        run = client.beta.threads.runs.retrieve(
            thread_id = thread.id,
            run_id= run.id
        )
        time.sleep(0.5)
    return run

In [ ]:
# run 객체를 대기 상태로 설정하고, 해당 스레드에서 실행을 완료할 때가지 기다림
run = wait_on_run(run, thread)

# status가 'complete'인 경우에는 결과를 출력합니다.
show_json(run)

### Message(메세지)

run이 완료되었다면, Assistant에 의해 처리된 결과를 보기 위해 Thread에서 messages를 확인할 수 있다.

In [ ]:
# thread.id 를 사용하여 메시지 목록을 가져옴
messages = client.beta.threads.messages.list(thread_id=thread.id)

#결과 출력
show_json(messages)

이전에 받은 답변을 기억하고 있는지 확인

In [ ]:
message = client.beta.threads.messages.create(
    thread_id= thread.id,
    role='user',
    content='더 간단하게 말해줄래?'
)

run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id= ASSISTANT_ID
)

# 답변 완료될때까지 대기
wait_on_run(run, thread)

# 마지막 사용자 메시지 이후 추가된 모든 메시지 검색
messages = client.beta.threads.messages.list(
    thread_id=thread.id, 
    order='asc', #오름차순으로 출력
    after=message.id  # 이전 메세지를 제외하고 출력
)
show_json(messages)

아미지 업로드와 함께 물어보기.

In [ ]:
file = client.files.create(
    file = open('brand_img/여기의성.png','rb'),
    purpose='vision'
)

message = client.beta.threads.messages.create(
    thread_id= thread.id,
    role='user',
    content=[
        {
            'type': 'text',
            'text': '이 상표의 의견서를 제시해주세요'
        },
        {
            'type': 'image_file',
            'image_file': {'file_id': file.id}
        }
    ],
    
)

run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id= ASSISTANT_ID,
    tools=[]
)

# 답변 완료될때까지 대기
wait_on_run(run, thread)

# 마지막 사용자 메시지 이후 추가된 모든 메시지 검색
messages = client.beta.threads.messages.list(
    thread_id=thread.id, 
    # order='asc', #오름차순으로 출력
    # after=message.id  # 이전 메세지를 제외하고 출력
)
show_json(messages)

In [ ]:

def submit_message(assistant_id, thread, user_message):
    #사용자 입력 메시지를 스레드에 추가
    client.beta.threads.messages.create(
        thread_id= thread.id,
        role = "user",
        content = user_message
    )

    #스레드에 메시지가 입력되었다면 실행 준비
    run= client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id
    )

    return run

def wait_on_run(run,thread):
    # run이 완료될대까지 기다림 : polling 하며 대기 (polling: 서버와 응답을 주고받음)
    while run.status == 'queued' or run.status == 'in_progress':
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id
        )
        time.sleep(0.5)
    return run

def get_response(thread):
    # 스레드에서 메세지 목록가져오기
    return client.beta.threads.messages.list(thread_id=thread.id, order='asc')


# 새로운 스레드 생성 및 메시지 제출 함수
def create_thread_and_run(user_input):
    # 사용자 입력을 받아 새로운 스래드를 생성하고, Assistant 에게 메시지를 제출
    thread= client.beta.threads.create()
    run = submit_message(ASSISTANT_ID, thread, user_input)
    return thread, run


In [ ]:
# 동시에 여러 요청을 처리하기 위해 스래드를 생성합니다.
thread1, run1 = create_thread_and_run('스타벅스 상표의 식별력은 어떤가요?')
thread2, run2 = create_thread_and_run('어떤 기준이 가장 중요한가요?')
thread3, run3 = create_thread_and_run('어떻게하면 식별력 있는 상표를 만들 수 있을까요?')

In [ ]:
import time  
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# 메시지 출력용 함수
def print_message(response):
    for res in response:
        print(f'[{res.role.upper()}]\n{res.content[0].text.value}\n')
    print("-" * 60)

#반복문에서 대기하는 함수
def wait_on_run(run, thread):
    while run.status == 'queued' or run.status == 'in_progress':
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id = run.id
        )
        time.sleep(0.5)
    return run




# 첫 번째 실행을 위해 대기
run1 = wait_on_run(run1, thread1)
print_message(get_response(thread1))

# 두 번째 실행을 위해 대기
run2 = wait_on_run(run2, thread2)
print_message(get_response(thread2))

# 세 번째 실행을 위해 대기
run3 = wait_on_run(run3, thread3)
# 세 번째 스레드를 마치면 감사 인사를 전하고 종료합니다 :)
run4 = submit_message(ASSISTANT_ID, thread3, "도와주셔서 감사합니다!")
run4 = wait_on_run(run4, thread3)
print_message(get_response(thread3))
